In [51]:
import sys
sys.path.append("/Users/rossg/Work/nanoparticles/pipeline")
#from openmoltools import openeye         # for Choderalab wrappers of openeye software
import openeye_mod as openeye             # personal modification of antechamber file generation function
import openeye.oechem as oechem
from openmoltools import packmol
import parmed as pmd
import subprocess
import os

# Preparation via tleap

Creating a set of functions to automate the setup.

In [77]:
def parametrize_mol(inname,resname):
    mol = OEGraphMol()           # initialising molecule object
    ifs = oemolistream()         # initialising input stream for reading in data
    ifs.SetFormat(OEFormat_MOL2) # specifying mol2 format input
    ifs.open(inname)           
    if OEReadMolecule(ifs,mol):  # this function automatically returns True or False, to help spot for errors. 
        pass
    else:
        print "Problem loading molecule!"
    mol = oechem.OEMol(mol)    
    oechem.OEAssignAromaticFlags(mol, oechem.OEAroModelOpenEye)    
    oechem.OEAddExplicitHydrogens(mol)
    mol.SetTitle(resname)
    if any([atom.GetName() == '' for atom in mol.GetAtoms()]):
        oechem.OETriposAtomNames(mol)
    openeye.oemol_to_antechamber(mol,resname+'_gaff.mol2',resname+'.frcmod',residue_name=resname)
    
def mol2pdb(inname):
    outname = inname.split('.')[0] + '.pdb'
    parm = pmd.load_file(inname,structure=False)
    parm.save(outname,overwrite=True)
    
def tleap_nanosim(drugname,dyename,boxname):
    tleap_input = """
    source oldff/leaprc.ff99SB
    source leaprc.gaff
    {0} = loadmol2 {0}_gaff.mol2
    {1} = loadmol2 {1}_gaff.mol2
    loadamberparams {0}.frcmod
    loadamberparams {1}.frcmod
    {2} = loadpdb {2}.pdb
    solvateBox box TIP3PBOX 0.5
    addIons2 box Na+ 0
    saveamberparm {2} out.prmtop out.inpcrd
    quit
    """.format(drugname,dyename,boxname)
    file_handle = open('tleap_commands', 'w')
    file_handle.writelines(tleap_input)
    file_handle.close()

    cmd = "tleap -f {0}".format(file_handle.name)
    subprocess.call(cmd,shell=True)
    
def nanopipeline(drugmol2,dyemol2,drugname='LIG',dyename='DYE',ndrugs=10,ndye=20,boxsize=50):
    """
    Starting with the mol2 files of drug and dye, the molecules are parametrised, assembled uniformly in 
    cubic box, and AMBER prmtop and inpcrd files are created for the system. 
    
    Parameters
    ----------
    drugmol2, dyemol2  : str, str
      the mol2 file name of the drug/dye, with or without hydrogens
    drugname, dyename  : str, str
      the residue name you want for the drug/dye residue name and output file naming schemes
    ndrugs/ndye: int
      the number of drug/dye molecules you want in the box
    boxsize: int
      the length of the cubic box in Angstroms
      
    Returns
    -------
    out.inpcrd and out.out.prmtop are created in the working directory, as well gaff output files
    """
    
    # Parmetrizing with antechamber via openeye:
    parametrize_mol(drugmol2,drugname)
    parametrize_mol(dyemol2,dyename)
    # Creating box of drugs and dye. Packmol requires PDB files
    mol2pdb(drugname+'_gaff.mol2')
    mol2pdb(dyename+'_gaff.mol2')
    box = packmol.pack_box([drugname+'_gaff.pdb',dyename+'_gaff.pdb'],[ndrugs,ndye], tolerance=2.0, box_size=boxsize)
    box.save('box.pdb')
    os.remove(drugname+'_gaff.pdb')
    os.remove(dyename+'_gaff.pdb')
    # Running tleap to create out.prmtop out.inpcrd for use with openmm
    tleap_nanosim(drugname,dyename,'box.pdb')
                            
def mixture_setup(molfiles,resnames,nmols,boxsize):
    """
    Unifinished function that generalises the function 'nanopipeline' for general mixtures.
    TODO: adapt 'tleap_nanosim' so that an arbitrary number of gaff files are loaded into tleap.
    """
    
    # Parmetrizing with antechamber via openeye:
    for i in range(len(molfiles)): parametrize_mol(molfiles[i],resname[i])
    # Creating box of drugs and dye. Packmol requires PDB files
    for i in range(len(molfiles)): mol2pdb(resnames[i]+'_gaff.mol2')
    molfiles_pdb = [mol + '_gaff.pdb' for mol in resnames]                   
    box = packmol.pack_box(molfiles_pdb, nmols, tolerance=2.0, box_size=boxsize)
    box.save('box.pdb')
    # Running tleap to create out.prmtop out.inpcrd for use with openmm
    tleap_nanosim(resnames,'box.pdb')                          

In [78]:
nanopipeline('1E.mol2','IR783.mol2')


# Mixture 

tolerance 2.000000
filetype pdb
output /var/folders/_2/bwj1fkzx78v92gc1lfvf8cv589gk45/T/tmpyPNmUB.pdb
add_amber_ter


structure LIG_gaff.pdb
  number 10 
  inside box 0. 0. 0. 50.000000 50.000000 50.000000
end structure

structure DYE_gaff.pdb
  number 20 
  inside box 0. 0. 0. 50.000000 50.000000 50.000000
end structure


# Mixture 

tolerance 2.000000
filetype pdb
output /var/folders/_2/bwj1fkzx78v92gc1lfvf8cv589gk45/T/tmpyPNmUB.pdb
add_amber_ter


structure LIG_gaff.pdb
  number 10 
  inside box 0. 0. 0. 50.000000 50.000000 50.000000
end structure

structure DYE_gaff.pdb
  number 20 
  inside box 0. 0. 0. 50.000000 50.000000 50.000000
end structure

